# Implementación del PSO gbest
Se comienza por importar las librerias con algunas utilidades como manejo de arreglos, numeros aleatorios y funciones de prueba

In [5]:
# Manejo de arreglos y operaciones matematicas
import numpy as np

# Importar funciones
import import_ipynb
from utils import FuncionesObjetivo as f

# Numeros aleatorios
import random

## Implementacion de funciones
Ahora se implementan funciones que se usaran en la implementacion del algoritmo

In [8]:
def pg_best(X, y, y_hat, X_ft, y_ft, y_hat_ft):
    """
    Esta funcion actualiza el personal best de cada particula y el global best
    - input: 
        - X: poblacion
        - y: personal best actual de cada particula
        - y_hat: global best
        - X_ft: fitness de la poblacion
        - y_ft: fitness del personal best
        - y_hat_ft: fitness del mejor global
    - output: personal best y global best actualizados
    """
    # Obtener el indice de las particulas donde su nueva posicion es mejor que su personal best
    idx_mejoresp = np.where(X_ft < y_ft)
    
    # Actualizar personal best
    y[idx_mejoresp] = X[idx_mejoresp]
    
    # Si se encuentra un valor mejor en la poblacion actualizada, se cambia el gbest
    if np.min(X_ft) < y_hat_ft:
        y_hat = X[[np.argmin(X_ft)]]
    
    return y, y_hat

def actualizar_vel(v, X, y, y_hat, c1, c2):
    """
    Esta funcion actualiza la velocidad de las particulas
    - input:
        - v: velocidad actual de las particulas
        - X: posicion actual de las particulas
        - y: personal best de cada particula
        - y_hat: global best
        - c1, c2: constantes para peso de la componente personal y social
    - output: velocidad actualizada de cada particula
    """
    # Calcular nueva velocidad
    v += c1 * random.random() * (y - X) + c2 * random.random() * (y_hat - X)
    
    return v

## Definir Hiperparametros del algoritmo

In [32]:
d = 4  # Numero de dimensiones
n = 100  # Numero de particulas
rango = [[-10, 10]]  # Rango de las variables
c1 = 0.5  # Peso para personal best
c2 = 0.5  # Peso social
funcion = f.f_colville  # funcion objetivo
max_gen = 500  # Maximo de generaciones
tol = 1e-3  # Tolerancia para detener el algoritmo

## Implementación del algoritmo

In [33]:
# Si solo hay un rango se considera el mismo rango para todas las variables
if len(rango) == 1:
    rango *= d
# Convertir rango a np.array
rango = np.array(rango)

# Definimos un mejor fitness grande para iniciar el algoritmo
mejor_fitness= np.infty
gen = 0

# Crear un vetor de soluciones para cada dimension con su rango dado
X = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))
    
# El personal best se inicializa con la poblacion ya que no ha habido actualizacion
y = X.copy() 

# Inicializar el global best con el minimo de la poblacion inicial
y_hat = X[[np.argmin(funcion(X))]]

# Incializar en cero la velocidad de las particulas
v = np.zeros_like(X)

# Seguir con el algoritmo mientra sean menos de 200 generaciones o el fitness sea menor a 0.001
while((mejor_fitness > tol) and (gen < max_gen)):
    # Calcular el fitness del personal best
    y_ft = funcion(y)
    
    # Fitness del mejor global
    y_hat_ft = funcion(y_hat)
    
    # Fitness de la poblacion
    X_ft = funcion(X)
    
    # Actualizar el personal y global best
    y, y_hat = pg_best(X, y, y_hat, X_ft, y_ft, y_hat_ft)
    
    # Actualizar velocidad
    v = actualizar_vel(v, X, y, y_hat, c1, c2)
    
    # Actualizar la posicion de las particulas
    X += v
    
    # Aumentar la generacion
    gen += 1
    
# Una vez concluido el algoritmo se muestran los resultados
print(f"El óptimo está en {np.around(y_hat[0], 3)} con un fitness de {funcion(y_hat)[0]:0.4f}")
print(f"Obtenido en la generación {gen}")

El óptimo está en [1.293 1.698 0.558 0.303] con un fitness de 0.5496
Obtenido en la generación 500


# Algo no está funcionando bien, los resultados no son los mejores